Download audio using pytube

In [4]:
from pytube import YouTube

# Enter the URL of the YouTube video
video_url = "https://www.youtube.com/watch?v=lks-x8ZM554"

# Create a YouTube object
yt = YouTube(video_url)

# Get the audio stream
audio_stream = yt.streams.filter(only_audio=True).first()

# Download the audio stream
audio_directory = "jeff dean leaders connect bangalore"
audio_stream.download(output_path=f"./{audio_directory}", filename="audio.mp4")

'/home/analyticalmonk/open_source/local_llm_stt_summarization/./jeff dean leaders connect bangalore/audio.mp4'

Generate transcript using Whisper

In [ ]:
# !pip install -U openai-whisper
# sudo apt update && sudo apt install ffmpeg

In [12]:
import whisper

model = whisper.load_model("base")

# Specify the path to the downloaded audio file
audio_file_path = f"./{audio_directory}/audio.mp4"

# Generate the transcript
transcript = model.transcribe(audio_file_path)
# transcript = model.transcribe("audio_sample.mp4")

# Print the transcript
print(transcript)


/home/analyticalmonk/miniconda3/envs/local_llm_stt_summarization/lib/python3.10/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


{'text': ' For example, is an example of of of using AI at the back end. I think as we as we go forward AI will touch every sector, every industry pretty much every use case.', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 5.0, 'text': ' For example, is an example of of of using AI at the back end.', 'tokens': [50364, 1171, 1365, 11, 307, 364, 1365, 295, 295, 295, 1228, 7318, 412, 264, 646, 917, 13, 50614], 'temperature': 0.0, 'avg_logprob': -0.2893678310305573, 'compression_ratio': 1.38135593220339, 'no_speech_prob': 0.041291121393442154}, {'id': 1, 'seek': 0, 'start': 5.0, 'end': 13.0, 'text': ' I think as we as we go forward AI will touch every sector, every industry pretty much every use case.', 'tokens': [50614, 286, 519, 382, 321, 382, 321, 352, 2128, 7318, 486, 2557, 633, 6977, 11, 633, 3518, 1238, 709, 633, 764, 1389, 13, 51014], 'temperature': 0.0, 'avg_logprob': -0.2893678310305573, 'compression_ratio': 1.38135593220339, 'no_speech_prob': 0.041291121393442154}], 'lan